In [1]:
from langchain.document_loaders import CSVLoader
local_doc_path = 'accidentLabeled.csv'
loader = CSVLoader(file_path=local_doc_path,encoding='utf-8')
docs = loader.load()

In [2]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings() 
query="accident at a crossroad"
embed = embeddings.embed_query(query)

/home/jyx/miniconda3/envs/learnlangchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
from langchain.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(docs, embeddings)
relative_docs = db.similarity_search(query)
relative_docs[:3]

[Document(page_content=': crossroad\naccident type: This crash case involves a seriously-injured, belt-restrained, 36-year-old, 35-week pregnant female driver of a 2016 Honda Accord four-door sedan, which was primarily involved in a frontal collision with the front plane of a 2008 Chrysler Town and Country minivan.This two-vehicle collision occurred during daylight hours on an asphalt, urban roadway. The northbound roadway consists of three travel lanes with the first travel lane as a dedicated right-turn lane onto the interstate on-ramp. The southbound roadway consists of three travel lanes with the third travel lane as a channelized left-turn lane onto the interstate on-ramp. The intersection is controlled by standard overhead, electric, tri-colored traffic signals. The roadway is straight, with a posted speed limit of 72 km/h (45 mph). At the time of the crash, the weather was clear and the roadway surface was dry.Vehicle 1, a 2016 Honda Accord sedan, was being driven southbound by 

In [4]:
from langchain.llms import OpenAI
context = "".join([doc.page_content for doc in relative_docs])
prompt=f"""You are an expert in generating accident reports. Please generate accident report about {query}, and here are some examples of relative accident reports: {context}"""

In [5]:
from openai import OpenAI
import os
openai_api_key = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=openai_api_key)
def get_completion_from_openai(prompt, model='gpt-3.5-turbo'):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.8,
        top_p=0.8,
        max_tokens=512,
        frequency_penalty=1.1,
        presence_penalty=1.1
    )
    return response.choices[0].message.content

In [6]:
from IPython.display import Markdown
import json
json_file = 'accident.json'
try:
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
except (FileNotFoundError, json.JSONDecodeError):
    data = []
for i in range(5):
    print(i)
    response = get_completion_from_openai(prompt=prompt)
    Markdown(response)
    new_data = {"query": query, "response": response}
    data.append(new_data)
    with open(json_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

0
1
2
3
4
